In [1]:
import matplotlib.pyplot as plt
import ruptures as rpt
import os
import matplotlib as mpl
import numpy as np
import local_models.local_models
from gpr_utils import *
import numpy as np
import itertools
import functools
import collections

import logging
import ml_battery.log

logger = logging.getLogger(__name__)

def consume(it):
    collections.deque(it, maxlen=0)

In [2]:
signal_subsample_rate = 20
SIGNAL_HZ = 52
HZ=SIGNAL_HZ/signal_subsample_rate
CHANNEL_COLS=list(range(1,4))
CHANNELS = len(CHANNEL_COLS)
LABEL_COL = 4

In [3]:
mpl.rcParams['figure.figsize'] = [16.0, 8.0]
mpl.rcParams['font.size'] = int(mpl.rcParams['figure.figsize'][1]*3)

In [4]:
project_dir = "/home/brown/disk2/chest_acc/pelt_training"
os.makedirs(project_dir, exist_ok=1)

In [5]:
pelt_dir = os.path.join(project_dir, "pelt")
gpr_dir = os.path.join(project_dir, "gpr")
os.makedirs(pelt_dir, exist_ok=1)
os.makedirs(gpr_dir, exist_ok=1)

In [6]:
data_dir = "/home/brown/disk2/USA/windowed_gaussian_process_study/data/chest_acc"
data_file_format = "{}.csv"

In [7]:
def F_hat(y,u,continuity_correction=False):
    # the empirical cumulative distribution function
    return np.sum(y[:,None]<u,axis=0)/y.shape[0] - (1/(2*y.shape[0]))*continuity_correction
    
def c_F_hat(y,sorted_full_y):
    fhats = []
    for channel in range(y.shape[-1]):
        fhats.append(F_hat(y[:,channel],sorted_full_y[:,channel])[1:-1])
    fhats = np.stack(fhats).T
    n_times_n_minus_l = np.arange(2,sorted_full_y.shape[0])
    n_times_n_minus_l *= sorted_full_y.shape[0] + 1 - n_times_n_minus_l
    numer = np.sum(fhats*np.ma.log(fhats).filled(0)+(1-fhats)*np.ma.log(1-fhats).filled(0),axis=1)
    return y.shape[0]*np.sum(numer/n_times_n_minus_l)
    
from ruptures.base import BaseCost

class NonparametricDistributionCost(BaseCost):

    # The 2 following attributes must be specified for compatibility.
    model = "F_hat"
    min_size = 2

    def fit(self, signal):
        """Set the internal parameter."""
        self.signal = signal
        sorted_signal = []
        for channel in range(signal.shape[-1]):
            sorted_signal.append(np.sort(self.signal[...,channel]))
        self.sorted_signal = np.stack(sorted_signal).T
        self.previous_end = None
        self.timelog = local_models.local_models.loggin.TimeLogger(
            logger=logger, 
            how_often=1, total=self.signal.shape[0]/5, 
            tag="pelting")
        return self

    def error(self, start, end):
        """Return the approximation cost on the segment [start:end].

        Args:
            start (int): start of the segment
            end (int): end of the segment

        Returns:
            float: segment cost
        """
        if self.previous_end == None:
            self.previous_end = end
        if end != self.previous_end:
            with self.timelog:
                self.previous_end = end
        cost = -c_F_hat(self.signal[start:end], self.sorted_signal)
        return cost

In [8]:
np.random.seed(0)
indices = list(range(1,16))
np.random.shuffle(indices)

In [9]:
train_set = indices[:int(len(indices)/2)]
test_set = indices[int(len(indices)/2):]

In [154]:
train_set, test_set

([2, 7, 9, 10, 15, 5, 3], [14, 11, 8, 12, 4, 1, 6, 13])

In [117]:
cvs = 2.**np.linspace(12,12,1)
rbfs = 2.**np.linspace(-1,1,1)*HZ
bws = np.linspace(80*HZ,100*HZ,2)
pelts = 2.**np.linspace(-4,-2,5)

In [148]:
cvs, rbfs, bws

(array([4096.]), array([1.3]), array([208., 260.]))

In [11]:
def peltify(dat, pen, filename):
    logger.info("{}".format(filename))
    peltified = rpt.Pelt(custom_cost=NonparametricDistributionCost()).fit(
        dat).predict(pen)
    np.savetxt(filename, peltified)
    
def peltify_fromdisk(data_file, gpr_file, pen, filename):
    dat1 = np.loadtxt(data_file,delimiter=",")
    subsampled_dat = dat1[::signal_subsample_rate]
    if gpr_file is None:
        return peltify(subsampled_dat[:,CHANNEL_COLS], pen, filename)
    dat2 = np.loadtxt(gpr_file)
    base, f = os.path.split(filename)
    peltify(dat2, pen, os.path.join(base, "gpr_" + f))
    peltify(np.concatenate((subsampled_dat[:,CHANNEL_COLS], dat2), axis=1), 
        pen, os.path.join(base, "both_" + f))
    
def peltify_fromq(q):
    import traceback
    cv, rbf, bw, pen, i = q.get()
    try:
        data_file = os.path.join(data_dir, data_file_format.format(i))
        if cv is not None:
            gpr_file = os.path.join(gpr_dir, 
                "c{:08.02f}_r{:08.02f}_b{:08.02f}_i{:02d}.dat".format(cv,rbf,bw,i))
            pelt_file = os.path.join(pelt_dir,
                "c{:08.02f}_r{:08.02f}_b{:08.02f}_p{:06.04f}_i{:02d}.pelt".format(cv,rbf,bw,pen,i))
        else:
            gpr_file = None
            pelt_file = os.path.join(pelt_dir,
                "raw_p{:06.04f}_i{:02d}.pelt".format(pen,i))
        logger.info("peltifying c{} r{} bw{} p{} i{}".format(cv,rbf,bw,pen,i))
        peltify_fromdisk(data_file, gpr_file, pen, pelt_file)
        return cv, rbf, bw, pen, i
    except Exception as e:
        logger.info(str(e))
        logger.info(traceback.format_exc())
    
def gprify(cv, rbf, bw, dat, change_points, gpr_filename):
    import traceback
    X = np.arange(dat.shape[0]).reshape(-1,1)
    index = local_models.local_models.ConstantDistanceSortedIndex(X.flatten())

    lm_kernel = local_models.local_models.TriCubeKernel(bandwidth=bw)
    kernel = np.sum((
        np.prod((
            gp.kernels.ConstantKernel(constant_value=cv, constant_value_bounds="fixed"),
            gp.kernels.RBF(length_scale=rbf, length_scale_bounds="fixed")
        )),
        gp.kernels.WhiteKernel(noise_level=cv, noise_level_bounds=[1e-10,1e10])
    ))
    regressor = GPR(kernel=kernel, normalize_y=True, n_restarts_optimizer=0, alpha=0)
    gpr_models = local_models.local_models.LocalModels(regressor)
    gpr_paramses=np.empty((X.shape[0], dat.shape[1]))
    for channel in range(dat.shape[1]):
        logger.info("{}, {}, {}, {}".format(channel, cv, rbf, bw))
        y = dat[:,channel]
        gpr_models.fit(X, y, index=index)
        try:
            gpr_paramses[:,channel] = gpr_models.transform(X,
                r=lm_kernel.support_radius()-1, weighted=True, kernel=lm_kernel,
                neighbor_beta0s=False, batch_size=int(X.shape[0]/10))[:,0]
        except Exception as e:
            logger.info(str(e))
            logger.info("transformation FAILED! {}".format(gpr_filename))
            logger.info(traceback.format_exc())
            return gpr_filename
        #logger.info("finished {}, {}, {}, {}".format(channel, cv, rbf, bw))
    logger.info("writing gpr {}".format(gpr_filename))
    np.savetxt(gpr_filename, gpr_paramses)
    return gpr_filename

def gprify_fromdisk(cv, rbf, bw, data_file, gpr_filename):
    dat = np.loadtxt(data_file,delimiter=",")
    subsampled_dat = dat[::signal_subsample_rate]
    change_points = np.nonzero(np.diff(subsampled_dat[:,LABEL_COL]))[0]
    gprify(cv, rbf, bw, subsampled_dat[:,CHANNEL_COLS], change_points, gpr_filename)
    
def gprify_fromq(q):
    cv, rbf, bw, i = q.get()
    logger.info("gprifying c{} r{} bw{} i{}".format(cv,rbf,bw,i))
    data_file = os.path.join(data_dir, data_file_format.format(i))
    gpr_file = os.path.join(gpr_dir, 
            "c{:08.02f}_r{:08.02f}_b{:08.02f}_i{:02d}.dat".format(cv,rbf,bw,i))
    gprify_fromdisk(cv, rbf, bw, data_file, gpr_file)
    return cv, rbf, bw, i


In [12]:
def gprq_2_peltq(inq, outq):
    cv, rbf, bw, i = gprify_fromq(inq)
    list(map(outq.put, ((cv,rbf,bw,pen,i) for pen in pelts)))

In [127]:
from multiprocessing import Pool, Queue, Manager

manager = Manager()

try: gpr_pool.terminate()
except: pass

gpr_pool = Pool(4, maxtasksperchild=1)
gpr_queue = manager.Queue()


/home/brown/python_packages/local_models/local_models/loggin.py:6: RuntimeWarning: divide by zero encountered in log10
  places = np.log10(np.abs(number))
/home/brown/python_packages/local_models/local_models/loggin.py:6: RuntimeWarning: divide by zero encountered in log10
  places = np.log10(np.abs(number))
/home/brown/python_packages/local_models/local_models/loggin.py:6: RuntimeWarning: divide by zero encountered in log10
  places = np.log10(np.abs(number))
/home/brown/python_packages/local_models/local_models/loggin.py:6: RuntimeWarning: divide by zero encountered in log10
  places = np.log10(np.abs(number))
/home/brown/python_packages/local_models/local_models/loggin.py:6: RuntimeWarning: divide by zero encountered in log10
  places = np.log10(np.abs(number))


In [146]:
import gc
gc.collect()

0

In [144]:
try: pelt_pool.terminate()
except: pass

pelt_pool = Pool(24, maxtasksperchild=1)

pelt_queue = manager.Queue()

/home/brown/python_packages/local_models/local_models/loggin.py:6: RuntimeWarning: divide by zero encountered in log10
  places = np.log10(np.abs(number))
/home/brown/python_packages/local_models/local_models/loggin.py:6: RuntimeWarning: divide by zero encountered in log10
  places = np.log10(np.abs(number))
/home/brown/python_packages/local_models/local_models/loggin.py:6: RuntimeWarning: divide by zero encountered in log10
  places = np.log10(np.abs(number))
/home/brown/python_packages/local_models/local_models/loggin.py:6: RuntimeWarning: divide by zero encountered in log10
  places = np.log10(np.abs(number))
/home/brown/python_packages/local_models/local_models/loggin.py:6: RuntimeWarning: divide by zero encountered in log10
  places = np.log10(np.abs(number))
/home/brown/python_packages/local_models/local_models/loggin.py:6: RuntimeWarning: divide by zero encountered in log10
  places = np.log10(np.abs(number))
/home/brown/python_packages/local_models/local_models/loggin.py:6: Run

In [145]:
len(pelts)*len(train_set)

35

In [168]:
raw_data_pelt_tasks = set([(None, None, None, pen, i) 
    for i in indices for pen in pelts])
done = set()

for task in raw_data_pelt_tasks:
    _,_,_,pen,i = task
    pelt_file = os.path.join(pelt_dir,
        "raw_p{:06.04f}_i{:02d}.pelt".format(pen,i))    
    if os.path.exists(pelt_file):
        done.add(task)

print(len(raw_data_pelt_tasks - done))


0


In [ ]:
for task in raw_data_pelt_tasks - done: 
    pelt_queue.put(task)
    
raw_pelt_task_results = pelt_pool.map_async(peltify_fromq, 
            [pelt_queue]*len(raw_data_pelt_tasks - done))

In [56]:
#raw_pelt_task_results_evaluated = list(map(lambda x: x.get(), raw_pelt_task_results))

In [147]:
pelt_queue.qsize()

0

In [ ]:
len(pelts), len(train_set)

In [150]:
gpr_tasks = set(itertools.product(cvs,rbfs,bws,indices))
done = set()

for task in gpr_tasks:
    c,r,b,i = task
    gpr_file = os.path.join(gpr_dir, 
        "c{:08.02f}_r{:08.02f}_b{:08.02f}_i{:02d}.dat".format(c,r,b,i)) 
    if os.path.exists(gpr_file):
        done.add(task)

print(len(gpr_tasks - done))

0


In [130]:
for task in (gpr_tasks-done):
    gpr_queue.put(task)

In [131]:
gpr_queue.qsize()

30

In [132]:
gpr_task_results = gpr_pool.map_async(gprify_fromq,
    [gpr_queue]*(len(gpr_tasks-done)))

In [149]:
pelts = pelts[len(pelts)-2:len(pelts)-1]
pelts

array([0.1767767])

In [183]:
gpr_pelt_task_results = pelt_pool.map_async(peltify_fromq,
    [pelt_queue]*(len(gpr_tasks)*len(pelts)))

In [184]:
def monitor_gpr_task():
    import time
    not_finished = set(itertools.product(cvs, rbfs, bws, pelts, test_set))
    done = set()

    for task in not_finished:
        pelt_file = os.path.join(pelt_dir,
            "c{:08.02f}_r{:08.02f}_b{:08.02f}_p{:06.04f}_i{:02d}.pelt".format(*task))    
        if os.path.exists(pelt_file):
            done.add(task)

    not_finished -= done

    gpr_files = set(os.listdir(gpr_dir))
    for cv, rbf, bw, pen, i in sorted(not_finished)[:100]:
        gpr_file = "c{:08.02f}_r{:08.02f}_b{:08.02f}_i{:02d}.dat".format(cv,rbf,bw,i)
        if gpr_file in gpr_files:
            pelt_queue.put((cv,rbf,bw,pen,i))


In [185]:
monitor_gpr_task()

In [149]:
kernel = np.sum((
    np.prod((
        gp.kernels.ConstantKernel(constant_value=0, constant_value_bounds="fixed"),
        gp.kernels.RBF(length_scale=0, length_scale_bounds="fixed")
    )),
    gp.kernels.WhiteKernel(noise_level=0, noise_level_bounds=[1e-10,1e10])
))
def plotify_gpr_params(gpr_filename, plt_filename):
    gpr_paramses = np.loadtxt(gpr_filename)
    X = np.arange(gpr_paramses.shape[0])
    plt_gpr_params(X, np.array([[]]).reshape(0,0), 
        X, gpr_paramses, 
        chg_ptses=[],
        filename=plt_filename, kernel=kernel, display=False)

In [151]:
gpr_files = set(os.listdir(gpr_dir))
plt_dir = os.path.join(project_dir, "plts")
os.makedirs(plt_dir, exist_ok=1)
for gpr_file in gpr_files:
    plt_file = os.path.join(plt_dir, gpr_file[:-4] + ".png")
    plotify_gpr_params(os.path.join(gpr_dir, gpr_file), plt_file)

In [155]:
def confuse(pelt, change_points, tol=1e1*HZ):
    pelt_expanded = pelt.reshape(-1,1).repeat(change_points.shape[0], axis=1)
    matches = (pelt_expanded >= change_points - tol) & (pelt_expanded <= change_points + tol)
    pelt_matches = np.any(matches, axis=0)
    gt_matches = np.any(matches, axis=1)
    tp = np.sum(pelt_matches)
    tn = np.sum(~gt_matches)
    fp = np.sum(~pelt_matches)
    return np.array([[np.nan, fp],[tn, tp]])

def confuse_fromdisk(pelt_file, dat_file, tol=1e1*HZ):
    pelt = np.loadtxt(pelt_file)
    dat = np.loadtxt(data_file,delimiter=",")
    subsampled_dat = dat[::signal_subsample_rate]
    change_points = np.nonzero(np.diff(subsampled_dat[:,LABEL_COL]))[0]
    return confuse(pelt, change_points, tol)

def precision(cm):
    return cm[1,1]/np.sum(cm[1])
def recall(cm):
    return cm[1,1]/np.sum(cm[:,1])
def f1(cm):
    p, r = precision(cm), recall(cm)
    return 2*(p*r)/(p+r)

In [169]:
train_cm_dict = collections.defaultdict(lambda: np.zeros((2,2)))
test_cm_dict = collections.defaultdict(lambda: np.zeros((2,2)))
for task in raw_data_pelt_tasks:
    _,_,_,pen,i = task
    pelt_file = os.path.join(pelt_dir,
        "raw_p{:06.04f}_i{:02d}.pelt".format(pen,i))
    data_file = os.path.join(data_dir, data_file_format.format(i))
    try:
        cm = confuse_fromdisk(pelt_file, data_file)
        if i in train_set:
            train_cm_dict[pen] += cm
        if i in test_set:
            test_cm_dict[pen] += cm
        print(i, pen, cm, precision(cm), recall(cm), f1(cm))
    except:
        print("not done", pen, i)

7 0.1767766952966369 [[nan  8.]
 [ 3.  1.]] 0.25 0.1111111111111111 0.15384615384615383
14 0.1767766952966369 [[nan  7.]
 [ 4.  2.]] 0.3333333333333333 0.2222222222222222 0.26666666666666666
2 0.1767766952966369 [[nan  8.]
 [ 4.  2.]] 0.3333333333333333 0.2 0.25
9 0.1767766952966369 [[nan  9.]
 [ 2.  1.]] 0.3333333333333333 0.1 0.15384615384615383
6 0.1767766952966369 [[nan 10.]
 [ 6.  1.]] 0.14285714285714285 0.09090909090909091 0.1111111111111111
13 0.1767766952966369 [[nan  7.]
 [ 3.  1.]] 0.25 0.125 0.16666666666666666
1 0.1767766952966369 [[nan  8.]
 [ 3.  1.]] 0.25 0.1111111111111111 0.15384615384615383
8 0.1767766952966369 [[nan  8.]
 [ 5.  3.]] 0.375 0.2727272727272727 0.3157894736842105
5 0.1767766952966369 [[nan  6.]
 [ 1.  3.]] 0.75 0.3333333333333333 0.46153846153846156
12 0.1767766952966369 [[nan  7.]
 [ 3.  1.]] 0.25 0.125 0.16666666666666666


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars


11 0.1767766952966369 [[nan  8.]
 [ 5.  0.]] 0.0 0.0 nan
15 0.1767766952966369 [[nan  8.]
 [ 6.  1.]] 0.14285714285714285 0.1111111111111111 0.125
4 0.1767766952966369 [[nan  7.]
 [ 5.  2.]] 0.2857142857142857 0.2222222222222222 0.25
3 0.1767766952966369 [[nan  7.]
 [ 3.  2.]] 0.4 0.2222222222222222 0.2857142857142857
10 0.1767766952966369 [[nan  7.]
 [ 4.  2.]] 0.3333333333333333 0.2222222222222222 0.26666666666666666


In [174]:
for pen in train_cm_dict:
    print(pen, train_cm_dict[pen], precision(train_cm_dict[pen]), recall(train_cm_dict[pen]), f1(train_cm_dict[pen]))

0.08838834764831845 [[0. 0.]
 [0. 0.]] nan nan nan
0.1767766952966369 [[nan 53.]
 [23. 12.]] 0.34285714285714286 0.18461538461538463 0.24
0.0625 [[0. 0.]
 [0. 0.]] nan nan nan
0.25 [[0. 0.]
 [0. 0.]] nan nan nan
0.125 [[0. 0.]
 [0. 0.]] nan nan nan


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars


In [175]:
train_cm_dict

defaultdict(<function __main__.<lambda>()>, {0.0625: array([[0., 0.],
                    [0., 0.]]), 0.08838834764831845: array([[0., 0.],
                    [0., 0.]]), 0.125: array([[0., 0.],
                    [0., 0.]]), 0.1767766952966369: array([[nan, 53.],
                    [23., 12.]]), 0.25: array([[0., 0.],
                    [0., 0.]])})

In [176]:
for pen in test_cm_dict:
    print(pen, test_cm_dict[pen], precision(test_cm_dict[pen]), recall(test_cm_dict[pen]), f1(test_cm_dict[pen]))

0.1767766952966369 [[nan 62.]
 [34. 11.]] 0.24444444444444444 0.1506849315068493 0.1864406779661017


In [177]:
train_set

[2, 7, 9, 10, 15, 5, 3]

In [178]:
m_cvs = 2.**np.linspace(8,10,3)
m_rbfs = 2.**np.linspace(-2,2,5)*HZ
m_bws = np.linspace(60*HZ,100*HZ,3)
m_pelts = 2.**np.linspace(-4,-2,5)
m_cvs = set(cvs) | set(m_cvs)
m_rbfs = set(rbfs) | set(m_rbfs)
m_bws = set(bws) | set(m_bws)
m_pelts = set(pelts) | set(m_pelts)

m_cvs, m_rbfs, m_bws, m_pelts

({256.0, 512.0, 1024.0, 4096.0},
 {0.65, 1.3, 2.6, 5.2, 10.4},
 {156.0, 208.0, 260.0},
 {0.0625, 0.08838834764831845, 0.125, 0.1767766952966369, 0.25})

In [189]:
train_gpr_cm_dict = collections.defaultdict(lambda: [np.zeros((2,2)),0])
train_both_cm_dict = collections.defaultdict(lambda: [np.zeros((2,2)),0])
test_gpr_cm_dict = collections.defaultdict(lambda: [np.zeros((2,2)),0])
test_both_cm_dict = collections.defaultdict(lambda: [np.zeros((2,2)),0])
#pelt_tasks = set(itertools.product(cvs, rbfs, bws, pelts, train_set))
pelt_tasks = set(itertools.product(m_cvs, m_rbfs, m_bws, m_pelts, indices))
for task in pelt_tasks:
    c,r,b,pen,i = task
    base_pelt_file = "c{:08.02f}_r{:08.02f}_b{:08.02f}_p{:06.04f}_i{:02d}.pelt".format(c,r,b,pen,i)
    gpr_pelt_file = os.path.join(pelt_dir, "gpr_" + base_pelt_file)
    both_pelt_file = os.path.join(pelt_dir, "both_" + base_pelt_file)
    data_file = os.path.join(data_dir, data_file_format.format(i))
    try:
        gpr_cm = confuse_fromdisk(gpr_pelt_file, data_file)
        if i in train_set:
            train_gpr_cm_dict[(c,r,b,pen)][0] += gpr_cm
            train_gpr_cm_dict[(c,r,b,pen)][1] += 1
        else:
            test_gpr_cm_dict[(c,r,b,pen)][0] += gpr_cm
            test_gpr_cm_dict[(c,r,b,pen)][1] += 1
        print(i, pen, gpr_cm, precision(gpr_cm), recall(gpr_cm), f1(gpr_cm))
    except Exception as e:
        print(e)
        print("gpr not done", pen, i)
    try:
        both_cm = confuse_fromdisk(both_pelt_file, data_file)
        if i in train_set:
            train_both_cm_dict[(c,r,b,pen)][0] += both_cm
            train_both_cm_dict[(c,r,b,pen)][1] += 1
        else:
            test_both_cm_dict[(c,r,b,pen)][0] += both_cm
            test_both_cm_dict[(c,r,b,pen)][1] += 1
        print(i, pen, both_cm, precision(both_cm), recall(both_cm), f1(both_cm))
    except:
        print("both not done", pen, i)

/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c01024.00_r00000.65_b00208.00_p0.0884_i02.pelt not found.
gpr not done 0.08838834764831845 2
both not done 0.08838834764831845 2
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00512.00_r00002.60_b00260.00_p0.0884_i13.pelt not found.
gpr not done 0.08838834764831845 13
both not done 0.08838834764831845 13
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00000.65_b00156.00_p0.0884_i04.pelt not found.
gpr not done 0.08838834764831845 4
both not done 0.08838834764831845 4
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c04096.00_r00002.60_b00208.00_p0.1768_i01.pelt not found.
gpr not done 0.1767766952966369 1
both not done 0.1767766952966369 1
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c01024.00_r00010.40_b00260.00_p0.0884_i13.pelt not found.
gpr not done 0.08838834764831845 13
both not done 0.08838834764831845 13
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c01024.00_r00010.40_b00260.00_p0.0625_i09.pelt 

2 0.1767766952966369 [[nan  8.]
 [ 7.  2.]] 0.2222222222222222 0.2 0.2105263157894737
2 0.1767766952966369 [[nan  7.]
 [ 9.  3.]] 0.25 0.3 0.2727272727272727
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00512.00_r00002.60_b00156.00_p0.0625_i15.pelt not found.
gpr not done 0.0625 15
both not done 0.0625 15
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c04096.00_r00002.60_b00208.00_p0.0884_i10.pelt not found.
gpr not done 0.08838834764831845 10
both not done 0.08838834764831845 10
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00010.40_b00208.00_p0.0625_i02.pelt not found.
gpr not done 0.0625 2
both not done 0.0625 2
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00010.40_b00208.00_p0.1250_i02.pelt not found.
gpr not done 0.125 2
both not done 0.125 2
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00010.40_b00208.00_p0.2500_i02.pelt not found.
gpr not done 0.25 2
both not done 0.25 2
/home/brown/disk2/chest_acc/pelt_training/pelt/g

2 0.1767766952966369 [[nan  9.]
 [ 7.  1.]] 0.125 0.1 0.11111111111111112
2 0.1767766952966369 [[nan  9.]
 [ 9.  1.]] 0.1 0.1 0.10000000000000002
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00010.40_b00156.00_p0.1250_i03.pelt not found.
gpr not done 0.125 3
both not done 0.125 3
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00010.40_b00156.00_p0.2500_i03.pelt not found.
gpr not done 0.25 3
both not done 0.25 3
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00512.00_r00005.20_b00156.00_p0.1768_i09.pelt not found.
gpr not done 0.1767766952966369 9
both not done 0.1767766952966369 9
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00512.00_r00005.20_b00260.00_p0.0625_i03.pelt not found.
gpr not done 0.0625 3
both not done 0.0625 3
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00512.00_r00005.20_b00260.00_p0.1250_i03.pelt not found.
gpr not done 0.125 3
both not done 0.125 3
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c04096.00_r00001.

/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00005.20_b00208.00_p0.0884_i03.pelt not found.
gpr not done 0.08838834764831845 3
both not done 0.08838834764831845 3
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00002.60_b00208.00_p0.1768_i02.pelt not found.
gpr not done 0.1767766952966369 2
both not done 0.1767766952966369 2
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00000.65_b00156.00_p0.0625_i03.pelt not found.
gpr not done 0.0625 3
both not done 0.0625 3
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00000.65_b00156.00_p0.1250_i03.pelt not found.
gpr not done 0.125 3
both not done 0.125 3
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00000.65_b00156.00_p0.2500_i03.pelt not found.
gpr not done 0.25 3
both not done 0.25 3
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00512.00_r00010.40_b00156.00_p0.0884_i08.pelt not found.
gpr not done 0.08838834764831845 8
both not done 0.08838834764831845 8
/home/

10 0.1767766952966369 [[nan  8.]
 [ 4.  1.]] 0.2 0.1111111111111111 0.14285714285714285
10 0.1767766952966369 [[nan  7.]
 [ 6.  2.]] 0.25 0.2222222222222222 0.23529411764705882
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00002.60_b00260.00_p0.1768_i02.pelt not found.
gpr not done 0.1767766952966369 2
both not done 0.1767766952966369 2
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c01024.00_r00001.30_b00208.00_p0.0884_i15.pelt not found.
gpr not done 0.08838834764831845 15
both not done 0.08838834764831845 15
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c04096.00_r00001.30_b00208.00_p0.0884_i01.pelt not found.
gpr not done 0.08838834764831845 1
both not done 0.08838834764831845 1
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00001.30_b00260.00_p0.0884_i06.pelt not found.
gpr not done 0.08838834764831845 6
both not done 0.08838834764831845 6
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00000.65_b00208.00_p0.0884_i04.pelt not f

...

7 0.1767766952966369 [[nan  8.]
 [ 6.  1.]] 0.14285714285714285 0.1111111111111111 0.125
both not done 0.1767766952966369 7
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00005.20_b00260.00_p0.1250_i10.pelt not found.
gpr not done 0.125 10
both not done 0.125 10
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c04096.00_r00002.60_b00208.00_p0.0884_i15.pelt not found.
gpr not done 0.08838834764831845 15
both not done 0.08838834764831845 15
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00005.20_b00260.00_p0.2500_i10.pelt not found.
gpr not done 0.25 10
both not done 0.25 10
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00010.40_b00208.00_p0.0625_i07.pelt not found.
gpr not done 0.0625 7
both not done 0.0625 7
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00010.40_b00208.00_p0.1250_i07.pelt not found.
gpr not done 0.125 7
both not done 0.125 7
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00256.00_r00010.40_b00208.00_p

10 0.1767766952966369 [[nan  7.]
 [ 5.  2.]] 0.2857142857142857 0.2222222222222222 0.25
10 0.1767766952966369 [[nan  7.]
 [ 9.  2.]] 0.18181818181818182 0.2222222222222222 0.19999999999999998
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c01024.00_r00000.65_b00208.00_p0.1250_i01.pelt not found.
gpr not done 0.125 1
both not done 0.125 1
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c01024.00_r00000.65_b00208.00_p0.2500_i01.pelt not found.
gpr not done 0.25 1
both not done 0.25 1
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c04096.00_r00005.20_b00208.00_p0.0625_i08.pelt not found.
gpr not done 0.0625 8
both not done 0.0625 8
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c04096.00_r00000.65_b00156.00_p0.0884_i14.pelt not found.
gpr not done 0.08838834764831845 14
both not done 0.08838834764831845 14
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c04096.00_r00005.20_b00208.00_p0.1250_i08.pelt not found.
gpr not done 0.125 8
both not done 0.125 8
/home/brown/disk2/

both not done 0.1767766952966369 8
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c01024.00_r00010.40_b00208.00_p0.0625_i07.pelt not found.
gpr not done 0.0625 7
both not done 0.0625 7
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c04096.00_r00000.65_b00208.00_p0.0884_i08.pelt not found.
gpr not done 0.08838834764831845 8
both not done 0.08838834764831845 8
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00512.00_r00001.30_b00260.00_p0.0884_i03.pelt not found.
gpr not done 0.08838834764831845 3
both not done 0.08838834764831845 3
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00512.00_r00005.20_b00156.00_p0.0625_i01.pelt not found.
gpr not done 0.0625 1
both not done 0.0625 1
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00512.00_r00005.20_b00156.00_p0.1250_i01.pelt not found.
gpr not done 0.125 1
both not done 0.125 1
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00512.00_r00005.20_b00156.00_p0.2500_i01.pelt not found.
gpr not done 0.25 1
both not done 0.

9 0.1767766952966369 [[nan  8.]
 [ 2.  2.]] 0.5 0.2 0.28571428571428575
both not done 0.1767766952966369 9
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00512.00_r00001.30_b00156.00_p0.1250_i08.pelt not found.
gpr not done 0.125 8
both not done 0.125 8
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00512.00_r00001.30_b00156.00_p0.2500_i08.pelt not found.
gpr not done 0.25 8
both not done 0.25 8
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00512.00_r00005.20_b00260.00_p0.0884_i06.pelt not found.
gpr not done 0.08838834764831845 6
both not done 0.08838834764831845 6
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c01024.00_r00001.30_b00156.00_p0.0625_i01.pelt not found.
gpr not done 0.0625 1
both not done 0.0625 1
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c01024.00_r00001.30_b00156.00_p0.1250_i01.pelt not found.
gpr not done 0.125 1
both not done 0.125 1
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c01024.00_r00001.30_b00156.00_p0.2500_i01.pelt not fou

2 0.1767766952966369 [[nan  9.]
 [ 7.  1.]] 0.125 0.1 0.11111111111111112
2 0.1767766952966369 [[nan  8.]
 [10.  2.]] 0.16666666666666666 0.2 0.1818181818181818
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00512.00_r00000.65_b00260.00_p0.1250_i03.pelt not found.
gpr not done 0.125 3
both not done 0.125 3
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00512.00_r00000.65_b00260.00_p0.2500_i03.pelt not found.
gpr not done 0.25 3
both not done 0.25 3
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c00512.00_r00001.30_b00260.00_p0.0625_i03.pelt not found.
gpr not done 0.0625 3
both not done 0.0625 3
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c01024.00_r00001.30_b00260.00_p0.0625_i04.pelt not found.
gpr not done 0.0625 4
both not done 0.0625 4
7 0.1767766952966369 [[nan  7.]
 [ 1.  2.]] 0.6666666666666666 0.2222222222222222 0.3333333333333333
both not done 0.1767766952966369 7
/home/brown/disk2/chest_acc/pelt_training/pelt/gpr_c01024.00_r00010.40_b00208.00_p0.0884_i01.

In [190]:
for c,r,b,pen in train_gpr_cm_dict:
    print(c,r,b,pen, f1(train_gpr_cm_dict[(c,r,b,pen)][0]), train_gpr_cm_dict[(c,r,b,pen)][1])

256.0 2.6 260.0 0.25 0.26666666666666666 1
256.0 10.4 208.0 0.0625 0.16666666666666669 1
256.0 10.4 156.0 0.0625 0.08333333333333334 1
512.0 10.4 156.0 0.1767766952966369 0.18556701030927836 7
256.0 10.4 208.0 0.125 0.125 1
256.0 10.4 208.0 0.1767766952966369 0.37499999999999994 1
256.0 2.6 208.0 0.25 0.13333333333333333 1
1024.0 0.65 156.0 0.1767766952966369 0.1518987341772152 5
256.0 2.6 208.0 0.125 0.11111111111111112 1
512.0 2.6 156.0 0.1767766952966369 0.13333333333333333 1
1024.0 2.6 156.0 0.1767766952966369 0.16666666666666666 4
256.0 0.65 208.0 0.1767766952966369 0.19469026548672566 7
512.0 0.65 156.0 0.0625 0.17391304347826086 1
256.0 2.6 260.0 0.125 0.22222222222222224 1
512.0 2.6 260.0 0.1767766952966369 0.2058823529411765 5
512.0 2.6 208.0 0.125 0.13333333333333333 1
1024.0 2.6 208.0 0.1767766952966369 0.14285714285714285 1
256.0 2.6 260.0 0.1767766952966369 0.26666666666666666 2
256.0 10.4 156.0 0.1767766952966369 0.13333333333333333 1
256.0 0.65 260.0 0.1767766952966369 0

In [191]:
for c,r,b,pen in test_gpr_cm_dict:
    print(c,r,b,pen, f1(test_gpr_cm_dict[(c,r,b,pen)][0]), test_gpr_cm_dict[(c,r,b,pen)][1])

4096.0 1.3 260.0 0.1767766952966369 0.16666666666666666 7
4096.0 1.3 208.0 0.1767766952966369 0.2051282051282051 6


In [188]:
for c,r,b,pen in both_cm_dict:
    print(c,r,b,pen, f1(both_cm_dict[(c,r,b,pen)][0]), both_cm_dict[(c,r,b,pen)][1])

256.0 10.4 208.0 0.0625 0.29268292682926833 1
256.0 10.4 260.0 0.1767766952966369 0.19999999999999998 1
512.0 10.4 156.0 0.1767766952966369 0.15789473684210525 4
256.0 10.4 208.0 0.125 0.17391304347826086 1
256.0 10.4 208.0 0.1767766952966369 0.3529411764705882 1
256.0 0.65 156.0 0.1767766952966369 0.16867469879518074 4
256.0 2.6 208.0 0.125 0.1818181818181818 1
512.0 2.6 156.0 0.1767766952966369 0.11764705882352941 1
1024.0 2.6 156.0 0.1767766952966369 0.18181818181818182 4
256.0 0.65 208.0 0.1767766952966369 0.19047619047619047 4
512.0 0.65 156.0 0.0625 0.21428571428571427 1
256.0 2.6 260.0 0.125 0.17391304347826086 1
512.0 2.6 260.0 0.1767766952966369 0.15 2
256.0 2.6 208.0 0.1767766952966369 0.25 1
1024.0 2.6 208.0 0.1767766952966369 0.23529411764705882 1
256.0 2.6 260.0 0.1767766952966369 0.2105263157894737 2
256.0 10.4 156.0 0.1767766952966369 0.1904761904761905 1
256.0 0.65 260.0 0.1767766952966369 0.19277108433734938 4
512.0 10.4 208.0 0.1767766952966369 0.15217391304347827 5
5

In [182]:
cvs

array([4096.])